# Playing with generator

In [2]:
#%%timeit
import glob
import os
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch
import numpy as np
import sys
import copy
from pathlib import Path
#from src.envs.generators import RandomRecurrence
#from src.envs.encoders import Equation, IntegerSeries, RealSeries
from sympy import *
import pickle
from collections import defaultdict, OrderedDict
import math

%load_ext autoreload
%autoreload 2
#%cd ~/recur/notebooks
sys.path.append('/private/home/sdascoli/recur')
import src
from src.envs.generators import RandomRecurrence
from src.envs.encoders import  Equation, FloatSequences, IntegerSequences
from src.envs.recurrence import RecurrenceEnvironment


In [3]:
class params:
    float_precision = 11
    max_len = 30
    min_len=10
    max_degree = 6
    max_number = 1e100
    float_sequences = True
    output_numeric = False
    max_exponent = 100
    init_scale = 10
    prob_const=1/3
    prob_n = 1/3
    max_int = 10
    prob_rand = 0.05
    dimension=1
    int_base=10000
    n_predictions=5
    float_tolerance=0.1
    more_tolerance="0.1"
    max_ops=10
    max_token_len=200
    eval_noise=False
    eval_input_length_modulo = -1
    operators_to_remove=""
    required_operators=""
    extra_unary_operators=""
    extra_binary_operators=""
    extra_constants=""
    float_constants=None
    use_sympy=False
    simplify=False
    mantissa_len = 3
    
generator = RandomRecurrence(params)
env = RecurrenceEnvironment(params)
env.rng = np.random.RandomState(0)
import time 

SPECIAL_WORDS = ["EOS", "PAD", "(", ")", "SPECIAL"]

if params.float_sequences:
    input_encoder = FloatSequences(params)
else:
    input_encoder = IntegerSequences(params)
input_words = SPECIAL_WORDS+sorted(list(set(input_encoder.symbols)))

if params.output_numeric:
    if params.float_sequences:
        output_encoder = FloatSequences(params)
    else:
        output_encoder = IntegerSequences(params)
else:
    output_encoder = Equation(params, generator.symbols)
print(env.output_word2id)
print(env.output_id2word)

deb = time.time()
k_trials=1000
successful_trials=0
size_input=[]
size_output=[]
infos_stats = {}
#n_ops_stats = {i:0 for i in range(1,(params.max_ops+1)*params.dimension)}

for k in range(k_trials):
    
    x, y, tree, infos = env.gen_expr(False, params.eval_input_length_modulo)
    if tree is not None:
        for info in infos:
            if info not in infos_stats:
                infos_stats[info]=np.zeros((1000,))
            for val in infos[info]:
                infos_stats[info][val]+=1

        successful_trials+=1
        size_input.append(len(x[0]))
        size_output.append(len(y[0]))

size_input=np.array(size_input)
size_output=np.array(size_output)
#n_valid_equations = sum(list(n_ops_stats.values()))
for info in infos_stats:
    non_null_points = np.where(infos_stats[info]!=0.)[0]
    values_non_null_points = infos_stats[info][non_null_points]
    count = values_non_null_points.sum()
    infos_stats[info] = np.concatenate([np.expand_dims(non_null_points,-1), np.expand_dims(values_non_null_points/count,-1)],-1)

print("Percentage of successful generation: {}".format(successful_trials/k_trials*100))
print("Time: {}".format(time.time()-deb))
print(infos_stats)
print("Average sizes: input {}, output {}".format(size_input.max(), size_output.max()))


{'-1': 0, '-10': 1, '-2': 2, '-3': 3, '-4': 4, '-5': 5, '-6': 6, '-7': 7, '-8': 8, '-9': 9, '0': 10, '1': 11, '10': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, 'INT+': 21, 'INT-': 22, 'abs': 23, 'add': 24, 'arccos': 25, 'arcsin': 26, 'arctan': 27, 'cos': 28, 'div': 29, 'e': 30, 'euler_gamma': 31, 'exp': 32, 'inv': 33, 'log': 34, 'mul': 35, 'n': 36, 'pi': 37, 'pow': 38, 'rand': 39, 'sin': 40, 'sqr': 41, 'sqrt': 42, 'sub': 43, 'tan': 44, 'x_0_0': 45, 'x_0_1': 46, 'x_0_2': 47, 'x_0_3': 48, 'x_0_4': 49, 'x_0_5': 50, 'x_0_6': 51, '|': 52, 'EOS': 53, 'PAD': 54, '(': 55, ')': 56, 'SPECIAL': 57, 'OOD_unary_op': 58, 'OOD_binary_op': 59, 'OOD_constant': 60, '': 60}
{0: '-1', 1: '-10', 2: '-2', 3: '-3', 4: '-4', 5: '-5', 6: '-6', 7: '-7', 8: '-8', 9: '-9', 10: '0', 11: '1', 12: '10', 13: '2', 14: '3', 15: '4', 16: '5', 17: '6', 18: '7', 19: '8', 20: '9', 21: 'INT+', 22: 'INT-', 23: 'abs', 24: 'add', 25: 'arccos', 26: 'arcsin', 27: 'arctan', 28: 'cos', 29: 'div', 30

/private/home/sdascoli/recur/src/envs/generators.py:158: RuntimeWarning: invalid value encountered in sqrt
  try: return getattr(np,self.value)(self.children[0].val(series))
/private/home/sdascoli/recur/src/envs/generators.py:158: RuntimeWarning: invalid value encountered in log
  try: return getattr(np,self.value)(self.children[0].val(series))
/private/home/sdascoli/recur/src/envs/generators.py:139: RuntimeWarning: divide by zero encountered in double_scalars
  return 1/(self.children[0].val(series))
/private/home/sdascoli/recur/src/envs/generators.py:122: RuntimeWarning: invalid value encountered in double_scalars
  return self.children[0].val(series) * self.children[1].val(series)
/private/home/sdascoli/recur/src/envs/generators.py:158: RuntimeWarning: invalid value encountered in arccos
  try: return getattr(np,self.value)(self.children[0].val(series))
/private/home/sdascoli/recur/src/envs/generators.py:158: RuntimeWarning: overflow encountered in exp
  try: return getattr(np,self.

Percentage of successful generation: 46.1
Time: 1.3668649196624756
{'n_input_points': array([[1.00000000e+01, 6.07375271e-02],
       [1.10000000e+01, 4.77223427e-02],
       [1.20000000e+01, 4.12147505e-02],
       [1.30000000e+01, 3.47071584e-02],
       [1.40000000e+01, 6.29067245e-02],
       [1.50000000e+01, 4.77223427e-02],
       [1.60000000e+01, 5.20607375e-02],
       [1.70000000e+01, 4.55531453e-02],
       [1.80000000e+01, 5.85683297e-02],
       [1.90000000e+01, 5.85683297e-02],
       [2.00000000e+01, 3.68763557e-02],
       [2.10000000e+01, 4.55531453e-02],
       [2.20000000e+01, 4.33839479e-02],
       [2.30000000e+01, 5.20607375e-02],
       [2.40000000e+01, 6.50759219e-02],
       [2.50000000e+01, 4.77223427e-02],
       [2.60000000e+01, 1.95227766e-02],
       [2.70000000e+01, 5.42299349e-02],
       [2.80000000e+01, 4.33839479e-02],
       [2.90000000e+01, 3.47071584e-02],
       [3.00000000e+01, 4.77223427e-02]]), 'n_ops': array([[ 1.        ,  0.11930586],
       

/private/home/sdascoli/recur/src/envs/generators.py:141: RuntimeWarning: overflow encountered in double_scalars
  return (self.children[0].val(series))**2


In [ ]:
python train.py --export_data True --dump_path=/private/home/pakamienny/Research_2/recur_datasets/all_domain/ --cpu True --max_epoch 2 --eval_input_length_modulo 5 --epoch_size 10000 

In [ ]:
x = np.array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 566.
             ])
np.argwhere(x!=0.)

# Checking encoder

In [32]:
x, y, tree, infos = env.gen_expr(False)
print(x,y,tree)

[['-', 'N9221', 'E-3', '-', 'N9586', 'E-3', '-', 'N4554', 'E-3', '-', 'N2530', 'E-3', '-', 'N7061', 'E-3', '+', 'N2104', 'E-3', '+', 'N1727', 'E-3', '+', 'N7105', 'E-3', '+', 'N9656', 'E-3', '+', 'N4364', 'E-3', '+', 'N1168', 'E-2', '+', 'N1099', 'E-2', '+', 'N1768', 'E-2', '+', 'N2035', 'E-2', '+', 'N1468', 'E-2', '+', 'N2242', 'E-2', '+', 'N2172', 'E-2', '+', 'N2851', 'E-2', '+', 'N3120', 'E-2', '+', 'N2547', 'E-2', '+', 'N3329', 'E-2', '+', 'N3258', 'E-2', '+', 'N3941', 'E-2', '+', 'N4210', 'E-2', '+', 'N3635', 'E-2', '+', 'N4420', 'E-2', '+', 'N4349', 'E-2', '+', 'N5033', 'E-2', '+', 'N5303', 'E-2']] [['sub', 'sub', 'add', 'n', 'abs', 'sqr', 'INT-', '3', 'div', 'INT+', '3', 'x_0_5', 'add', 'INT-', '2', 'sub', 'n', 'x_0_5']] (((n add abs((-3)**2)) sub (3 div x_0_5)) sub (-2 add (n sub x_0_5)))


In [33]:
encoder = Equation(params, generator.symbols)
print(tree.infix())
encoded = encoder.encode(tree)
print(encoded)
decoded = encoder.decode(encoded)
print(decoded.infix())

(((n add abs((-3)**2)) sub (3 div x_0_5)) sub (-2 add (n sub x_0_5)))
['sub', 'sub', 'add', 'n', 'abs', 'sqr', 'INT-', '3', 'div', 'INT+', '3', 'x_0_5', 'add', 'INT-', '2', 'sub', 'n', 'x_0_5']
(((n add abs((-3)**2)) sub (3 div x_0_5)) sub (-2 add (n sub x_0_5)))


In [9]:
x = [(-1)**n*10**-n for n in range(130)]
encoder = FloatSequences(params)
#print(encoder.symbols)
print(x)
encoded = encoder.encode(x)
print(encoded)
decoded = encoder.decode(encoded)
print(decoded)

[1, -0.1, 0.01, -0.001, 0.0001, -1e-05, 1e-06, -1e-07, 1e-08, -1e-09, 1e-10, -1e-11, 1e-12, -1e-13, 1e-14, -1e-15, 1e-16, -1e-17, 1e-18, -1e-19, 1e-20, -1e-21, 1e-22, -1e-23, 1e-24, -1e-25, 1e-26, -1e-27, 1e-28, -1e-29, 1e-30, -1e-31, 1e-32, -1e-33, 1e-34, -1e-35, 1e-36, -1e-37, 1e-38, -1e-39, 1e-40, -1e-41, 1e-42, -1e-43, 1e-44, -1e-45, 1e-46, -1e-47, 1e-48, -1e-49, 1e-50, -1e-51, 1e-52, -1e-53, 1e-54, -1e-55, 1e-56, -1e-57, 1e-58, -1e-59, 1e-60, -1e-61, 1e-62, -1e-63, 1e-64, -1e-65, 1e-66, -1e-67, 1e-68, -1e-69, 1e-70, -1e-71, 1e-72, -1e-73, 1e-74, -1e-75, 1e-76, -1e-77, 1e-78, -1e-79, 1e-80, -1e-81, 1e-82, -1e-83, 1e-84, -1e-85, 1e-86, -1e-87, 1e-88, -1e-89, 1e-90, -1e-91, 1e-92, -1e-93, 1e-94, -1e-95, 1e-96, -1e-97, 1e-98, -1e-99, 1e-100, -1e-101, 1e-102, -1e-103, 1e-104, -1e-105, 1e-106, -1e-107, 1e-108, -1e-109, 1e-110, -1e-111, 1e-112, -1e-113, 1e-114, -1e-115, 1e-116, -1e-117, 1e-118, -1e-119, 1e-120, -1e-121, 1e-122, -1e-123, 1e-124, -1e-125, 1e-126, -1e-127, 1e-128, -1e-129]


In [30]:
num=1000
base = 5
f"%0{base}d" % num

'01000'